# Traffic sign detection and classification

In [1]:
from xml.dom import minidom
from os import walk
import pandas as pd
import cv2 as cv
import numpy as np
from sklearn import metrics
import os


In [221]:
filenames = next(walk("res/annotations"), (None, None, []))[2]  # [] if no file

signs = []

for annotation in filenames:
    # parse an xml file by name
    file = minidom.parse("res/annotations/" + annotation)

    #use getElementsByTagName() to get tag
    if len(file.getElementsByTagName('name')) == 1:
        path = "res/images/" + file.getElementsByTagName('filename')[0].firstChild.data
        filename = file.getElementsByTagName('filename')[0].firstChild.data
        name = file.getElementsByTagName('name')[0].firstChild.data
    # truncated = file.getElementsByTagName('truncated')[0].firstChild.data
    # occluded = file.getElementsByTagName('occluded')[0].firstChild.data
    # difficult = file.getElementsByTagName('difficult')[0].firstChild.data

    if name == "trafficlight":
        continue

    signs.append([filename, name, path])

print(f'Importing {len(signs)} images')

df = pd.DataFrame(signs, columns=['filename', 'name', 'path'])


Importing 818 images


In [222]:
def condition_classes(s):
    if s['name'] == 'speedlimit':
        return 0
    elif s["name"] == 'crosswalk':
        return 1
    elif s["name"] == "stop":
        return 2


df["class"] = df.apply(condition_classes, axis=1)

## Step 1 - Histogram equalization

TODO - Need to improve the histogram equalization

In [223]:
def apply_histogram_equalization(row):
    img = cv.imread(row.path)
    lab = cv.cvtColor(img, cv.COLOR_BGR2LAB)

    clahe = cv.createCLAHE(clipLimit=10.0,tileGridSize=(8,8))

    lab[...,0] = clahe.apply(lab[...,0])

    out = cv.cvtColor(lab, cv.COLOR_LAB2BGR)
    
    #para ignorar este passo 
    out = img

    cv.imwrite("output2/1histogram/" + row.filename, out)

In [224]:
df.apply(apply_histogram_equalization, axis=1)

0      None
1      None
2      None
3      None
4      None
       ... 
813    None
814    None
815    None
816    None
817    None
Length: 818, dtype: object

## Step 2 - Segmentation by Color

In [225]:
def apply_segmentation(row):
    img = cv.imread(row.path)
    # TODO - work on histogram equalization
    img_hist = cv.imread("output2/1histogram/" + row.filename)
    img_hsv = cv.cvtColor(img_hist, cv.COLOR_BGR2HSV)

    lower_red_m1 = (0, 70, 60)
    upper_red_m1 = (10, 255, 255)

    lower_red_m2 = (170, 70, 60)
    upper_red_m2 = (180, 255, 255)

    lower_blue_m3 = (94, 127, 20)
    upper_blue_m3 = (126, 255, 200)

    mask1 = cv.inRange(img_hsv, lower_red_m1, upper_red_m1)
    mask2 = cv.inRange(img_hsv, lower_red_m2, upper_red_m2)
    blue_mask = cv.inRange(img_hsv, lower_blue_m3, upper_blue_m3)

    red_mask = mask1 + mask2

    # out = cv.bitwise_and(img_hist, img_hist, mask=mask)
    if cv.countNonZero(red_mask) !=0 :
        cv.imwrite("output2/2segmentation/red/" + row.filename, red_mask)
    if cv.countNonZero(blue_mask) != 0:
        cv.imwrite("output2/2segmentation/blue/" + row.filename, blue_mask)


In [226]:
df.apply(apply_segmentation, axis=1)

0      None
1      None
2      None
3      None
4      None
       ... 
813    None
814    None
815    None
816    None
817    None
Length: 818, dtype: object

## Step 3 - Post-Processing

In [227]:
def compare_masks(blue, red): 
    return blue if red == np.zeros(red.shape) else (red if blue == np.zeros(blue.shape) else blue, red)

In [273]:
def apply_post_processing_blue(row):
    img = cv.imread(row.path)
    segm_img = cv.imread("output2/2segmentation/blue/" + row.filename, cv.IMREAD_GRAYSCALE)
    if cv.countNonZero(segm_img) == 0 :
        print(row)
        return

    linesP = cv.HoughLinesP(segm_img, 1, np.pi / 180, 50, None, 50, 10)

    if linesP is not None:
        for i in range(0, len(linesP)):
            l = linesP[i][0]
            cv.line(segm_img, (l[0], l[1]), (l[2], l[3]),
                    (255, 0, 255), 3, cv.LINE_AA)

    # blue 
    # apply median filter to remove noise
    out = cv.medianBlur(segm_img, 5)
    rows, cols = out.shape

    # Taking a matrix of size 5 as the kernel
    kernel = np.ones((5, 5), np.uint8)

    # morphological operations
    out = cv.morphologyEx(out, cv.MORPH_CLOSE, kernel, iterations=4)

    # remove small and weird objects
    contours, hierarchy = cv.findContours(out, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)[-2:]
    for contour in contours:
        x, y, w, h = cv.boundingRect(contour)
        aspect_ratio = float(w) / h
        if cv.contourArea(contour) < 1 / 1500.0 * rows * cols and (aspect_ratio > 0.5 or aspect_ratio < 1.3):
            out = cv.fillPoly(out, pts=contour, color=(0, 0, 0))

    mask = np.full(img.shape, 0, "uint8")
    contours, hierarchies = cv.findContours(out, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)[-2:]
    for cnt in contours:
        cv.drawContours(mask, [cnt], -1, (255, 255, 255), -1)

    mask = cv.cvtColor(mask, cv.COLOR_BGR2GRAY)
    
    # morphological operations
    out = cv.erode(mask, kernel, iterations=1)
    out = cv.dilate(mask, kernel, iterations=5)

    cv.imwrite("output2/3post_processing/blue/" + row.filename, out)


In [274]:
df.apply(apply_post_processing_blue, axis=1)

filename                     road103.png
name                          speedlimit
path              res/images/road103.png
class                                  0
shapes                                []
classification                        -1
Name: 3, dtype: object
filename                     road104.png
name                          speedlimit
path              res/images/road104.png
class                                  0
shapes                                []
classification                        -1
Name: 4, dtype: object
filename                     road108.png
name                          speedlimit
path              res/images/road108.png
class                                  0
shapes                                []
classification                        -1
Name: 8, dtype: object
filename                     road109.png
name                          speedlimit
path              res/images/road109.png
class                                  0
shapes                       

0      None
1      None
2      None
3      None
4      None
       ... 
813    None
814    None
815    None
816    None
817    None
Length: 818, dtype: object

In [230]:
# def apply_post_processing(row):
#     img = cv.imread(row.path)
#     blue_segm_img = cv.imread(
#         "output2/2segmentation/blue/" + row.filename, cv.IMREAD_GRAYSCALE)
#     red_segm_img = cv.imread(
#         "output2/2segmentation/red/" + row.filename, cv.IMREAD_GRAYSCALE)


#     # blue
#     # apply median filter to remove noise
#     out = cv.medianBlur(segm_img, 5)
#     rows, cols = out.shape

#     # Taking a matrix of size 5 as the kernel
#     kernel = np.ones((5, 5), np.uint8)

#     # morphological operations
#     out = cv.morphologyEx(out, cv.MORPH_CLOSE, kernel, iterations=4)

#     # remove small and weird objects
#     contours, hierarchy = cv.findContours(
#         out, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)[-2:]
#     for contour in contours:
#         x, y, w, h = cv.boundingRect(contour)
#         aspect_ratio = float(w) / h
#         if cv.contourArea(contour) < 1 / 1500.0 * rows * cols and (aspect_ratio > 0.5 or aspect_ratio < 1.3):
#             out = cv.fillPoly(out, pts=contour, color=(0, 0, 0))

#     mask = np.full(img.shape, 0, "uint8")
#     contours, hierarchies = cv.findContours(
#         out, cv.RETR_TREE, cv.CHAIN_APPROX_NONE)[-2:]
#     for cnt in contours:
#         cv.drawContours(mask, [cnt], -1, (255, 255, 255), -1)

#     mask = cv.cvtColor(mask, cv.COLOR_BGR2GRAY)
#     # morphological operations
#     out = cv.erode(mask, kernel, iterations=1)
#     out = cv.dilate(mask, kernel, iterations=1)

#     cv.imwrite("output2/3post_processing/" + row.filename, out)


In [231]:
# df.apply(apply_post_processing, axis=1);

## Step 4 - Shape Recognition

In [275]:
## TODO MERGE DAS SHAPES DAS ROWS 
## IR AS DUAS PASTAS E FAZER append das shapes que dão

def shape_recognition(row):
    img = cv.imread(row.path)
    processed = cv.imread("output2/3post_processing/blue/" + row.filename, 0)

    _, thresh = cv.threshold(processed, 240, 255, cv.CHAIN_APPROX_SIMPLE)
    contours, _ = cv.findContours(
        thresh, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)[-2:]

    contours = sorted(contours, key=lambda x: -cv.contourArea(x))[:10]

    shapes = []

    for contour in contours:
        if float(cv.contourArea(contour) / (img.shape[0]*img.shape[1])) >= 0.95:
            continue
        approx = cv.approxPolyDP(contour, 0.01*cv.arcLength(contour, True), True)
        if len(approx) == 4:
            shapes.append(("rectangle", cv.contourArea(contour), (cv.boundingRect(contour))))
        elif len(approx) == 8:
            shapes.append(("octagon", cv.contourArea(contour), (cv.boundingRect(contour))))
        elif len(approx) > 8:
            shapes.append(("circle", cv.contourArea(contour), (cv.boundingRect(contour))))

    return shapes


In [276]:
df["shapes"] = df.apply(shape_recognition, axis=1);

In [277]:

def classify(row):
    shapes_info = row.shapes
    if len(shapes_info) == 0:
        return -1

    shapes = [x[0] for x in shapes_info]
    shape = shapes[0]

    if shape == 'circle':
        return 0
    elif shape == 'rectangle':
        return 1
    elif shape == 'octagon':
        return 2

    return -1

df["classification"] = df.apply(classify, axis=1);

In [278]:
def concatenate_and_write(row):
    img = cv.imread(row.path)
    #hist = cv.imread("output2/1histogram/red/" + row.filename)
    hist = img
    segm = cv.imread("output2/2segmentation/blue/" + row.filename)
    post = cv.imread("output2/3post_processing/blue/" + row.filename)
    vis = np.concatenate((img, hist, segm, post), axis=1)

    result = "right" if row['class'] == row['classification'] else "wrong"

    c = df['class'][row.name]
    p = os.path.join("output2", "4concatenated", "blue",
                     f'classe{c}', result + "_" + row.filename)

    cv.imwrite(p, vis)


In [279]:
df.apply(concatenate_and_write, axis=1)

0      None
1      None
2      None
3      None
4      None
       ... 
813    None
814    None
815    None
816    None
817    None
Length: 818, dtype: object

## Results

In [237]:
df_found = df[df["classification"] != -1]

accuracy_total = metrics.accuracy_score(df["class"], df["classification"])
accuracy_found = metrics.accuracy_score(df_found["class"], df_found["classification"])

print("Classes:\n", df["class"].value_counts())
print("Detected classes:\n", df["classification"].value_counts())
print("Accuracy Total: {:.02f}%".format(accuracy_total*100))
print("Accuracy Signs Found: {:.02f}%".format(accuracy_found*100))

Classes:
 0    620
2    145
1     53
Name: class, dtype: int64
Detected classes:
  0    377
-1    336
 2     53
 1     52
Name: classification, dtype: int64
Accuracy Total: 45.11%
Accuracy Signs Found: 76.56%


In [281]:
failed_0 = len(df[(df["class"] == 0) & (df["classification"] != 0)])    # speedlimit
failed_1 = len(df[(df["class"] == 1) & (df["classification"] != 1)])    # crosswalk
failed_2 = len(df[(df["class"] == 2) & (df["classification"] != 2)])    # stop

failed_1_0 = len(df[(df["class"] == 1) & (df["classification"] == 0)])
failed_1_2 = len(df[(df["class"] == 1) & (df["classification"] == 2)])

right_0 = len(df[(df["class"] == 0) & (df["classification"] == 0)])
right_1 = len(df[(df["class"] == 1) & (df["classification"] == 1)])
right_2 = len(df[(df["class"] == 2) & (df["classification"] == 2)])

not_classified_0 = len(df[(df["class"] == 0) & (df["classification"] == -1)])
not_classified_1 = len(df[(df["class"] == 1) & (df["classification"] == -1)])
not_classified_2 = len(df[(df["class"] == 2) & (df["classification"] == -1)])

print("Class 0 - SpeedLimit: R:{}|W:{}|NC:{}".format(right_0, failed_0, not_classified_0))
print("Class 1 - Crosswalk: R:{}|W:{}|NC:{}|0:{}|2:{}".format(right_1, failed_1, not_classified_1, failed_1_0,failed_1_2))
print("Class 2 - Stop: R:{}|W:{}|NC:{}".format(right_2, failed_2, not_classified_2))


Class 0 - SpeedLimit: R:297|W:323|NC:227
Class 1 - Crosswalk: R:16|W:37|NC:19|0:12|2:6
Class 2 - Stop: R:0|W:145|NC:126


In [4]:
def apply_post_processing_blue(img):
    segm_img = cv.imread("output2\\2segmentation\\blue\\road140.png", 0)
    cv.imshow("out", segm_img)
    cv.waitKey(0)
    cv.destroyWindow("out")

    linesP = cv.HoughLinesP(segm_img, 1, np.pi / 180, 50, None, 60, 20)

    if linesP is not None:
        for i in range(0, len(linesP)):
            l = linesP[i][0]
            cv.line(segm_img, (l[0], l[1]), (l[2], l[3]),
                    (255, 0, 255), 3, cv.LINE_AA)
   
    # blue
    # apply median filter to remove noise
    out = cv.medianBlur(segm_img, 5)
    
    cv.imshow("blur", out)
    cv.waitKey(0)
    cv.destroyWindow("blur")

    rows, cols = out.shape

    # Taking a matrix of size 5 as the kernel
    kernel = np.ones((5, 5), np.uint8)

    # morphological operations
    out = cv.morphologyEx(out, cv.MORPH_CLOSE, kernel, iterations=4)


    # remove small and weird objects
    contours, hierarchy = cv.findContours(
        out, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)[-2:]
    for contour in contours:
        x, y, w, h = cv.boundingRect(contour)
        aspect_ratio = float(w) / h
        if cv.contourArea(contour) < 1 / 1500.0 * rows * cols and (aspect_ratio > 0.5 or aspect_ratio < 1.3):
            out = cv.fillPoly(out, pts=contour, color=(0, 0, 0))

    cv.imshow("out", out)
    cv.waitKey(0)
    cv.destroyWindow("out")

    mask = np.full(segm_img.shape, 0, "uint8")
    contours, hierarchies = cv.findContours(
        out, cv.RETR_TREE, cv.CHAIN_APPROX_NONE)[-2:]
    for cnt in contours:
        cv.drawContours(mask, [cnt], -1, (255, 255, 255), -1)


    # morphological operations
    out = cv.erode(mask, kernel, iterations=1)
    out = cv.dilate(mask, kernel, iterations=6)

    cv.imshow("out", out)
    cv.waitKey(0)
    cv.destroyWindow("out")

    cv.imwrite("output2/3post_processing/blue/road140.png", out)


apply_post_processing_blue("ss")


In [7]:
def shape_recognition():
    processed = cv.imread("output2/3post_processing/blue/road140.png", 0)
    cv.imshow("out", processed)
    cv.waitKey(0)
    cv.destroyWindow("out")


    _, thresh = cv.threshold(processed, 240, 255, cv.CHAIN_APPROX_SIMPLE)
    contours, _ = cv.findContours(
        thresh, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)[-2:]

    contours = sorted(contours, key=lambda x: -cv.contourArea(x))[:10]

    shapes = []

    for contour in contours:
        if float(cv.contourArea(contour) / (processed.shape[0]*processed.shape[1])) >= 0.95:
            continue
        approx = cv.approxPolyDP(
            contour, 0.01*cv.arcLength(contour, True), True)
        cv.drawContours(processed, [contour], 0, (128, 0, 255), 2)
        print(len(approx))
        cv.imshow("out", processed)
        cv.waitKey(0)
        cv.destroyWindow("out")
        
        if len(approx) == 4:
            shapes.append(("rectangle", cv.contourArea(
                contour), (cv.boundingRect(contour))))
        elif len(approx) == 8:
            shapes.append(("octagon", cv.contourArea(
                contour), (cv.boundingRect(contour))))
        elif len(approx) > 8:
            shapes.append(("circle", cv.contourArea(
                contour), (cv.boundingRect(contour))))

    return shapes

shape_recognition()


10
6
11
5
4


[('circle', 5697.0, (292, 132, 69, 109)),
 ('circle', 3232.0, (209, 163, 70, 62)),
 ('rectangle', 312.0, (386, 214, 14, 25))]